In [80]:
# import libraries
import pandas as pd
import geopandas as gpd

## Load file

In [81]:
df = gpd.read_file("./CapasEditadas/EspaciosPublicos_v11.geojson")
df['evp'] = 0

# EP radius
df.loc[df.tipo == 'Plazoleta', 'evp'] = 1
df.loc[df.tipo == 'Plaza', 'evp'] = 1
df.loc[df.tipo == 'Parque', 'evp'] = 1
df.loc[df.tipo == 'Parque con equipamiento', 'evp'] = 1

df_ep = df.loc[(df.limite == 1)&(df.evp == 1)].reset_index(drop=True)
df_ep.head(3)

,full_id,fuente,distrito,evp,tipo,nombre,limite,area_m2,sis_bool,sis_nam,sis_area_m2,barpop_cant,barpop_nam,bus50_cant,bus500_cant,observaciones,geometry
0,r13232486,OSM,Lomas de Zamora,1,Plaza,Plaza Carlos Pellegrini,1,6932.85,0,None,NaN,0.0,None,0.0,7.0,None,"MULTIPOLYGON (((-58.42685 -34.79316, -58.42684..."
1,w28051684,OSM,Lomas de Zamora,1,Plaza,"Plaza Libertad Y Democracia, Dr. Raúl Alfonsín",1,14263.38,0,None,NaN,0.0,None,0.0,2.0,None,"MULTIPOLYGON (((-58.41578 -34.75867, -58.41435..."
2,w28051696,OSM,Lomas de Zamora,1,Plaza,Plaza de los Italianos,1,7883.71,0,None,NaN,0.0,None,0.0,9.0,None,"MULTIPOLYGON (((-58.42691 -34.76192, -58.42614..."


In [82]:
# importar capas de municipios y límite antrópico
gdf_muni = gpd.read_file("../1_Datos/ACUMAR/LIMITE_POLITICO.shp")
gdf_antro = gpd.read_file("../1_Datos/ACUMAR/LIMITE_ANTROPICO_POLIGONO.geojson")
# filtro de los municipios de interés 
gdf_muni = gdf_muni.loc[(gdf_muni.DISTRITO == "LOMAS DE ZAMORA") | (gdf_muni.DISTRITO == "LANUS")]

In [83]:
# importar capas de localidades
gdf_local = gpd.read_file("../1_Datos/Zonificación_Localidades_L y LZ.geojson")

# number and area by distrito

In [84]:
df_ep.groupby('distrito')['area_m2'].agg(['size', 'sum']).reset_index()

,distrito,size,sum
0,Lanús,107,840267.29
1,Lomas de Zamora,70,7148687.64


# number and area by localidad

In [117]:
# clip localidades
df_dif = gpd.overlay(gdf_local, gdf_antro[['geometry']], how='intersection')
#df_dif_sj = gpd.overlay(df_dif, df_ep[['full_id', 'area_m2', 'geometry']], how='intersection')
#df_dif_sj.area_m2 = df_dif_sj.to_crs("EPSG:5347").area
df_dif_sj = gpd.sjoin(df_dif, df_ep[['full_id', 'area_m2', 'geometry']])
df_dif_sj.drop_duplicates(subset="full_id", inplace=True)

In [118]:
df_dif_data = df_dif_sj.groupby('localidad')['area_m2'].agg(['size', 'sum']).reset_index()
gdf_local_data = pd.merge(df_dif, df_dif_data, on='localidad', how='left')
gdf_local_data.to_file("./Indice_m2/EP_localidad_antro.geojson", driver='GeoJSON')
gdf_local_data[['municipio', 'localidad', 'size', 'sum']].to_csv("./Indice_m2/EP_localidad_antro.csv")

In [120]:
gdf_local_data[['municipio', 'localidad', 'size', 'sum']]

,municipio,localidad,size,sum
0,Lanús,Valentín Alsina,38.0,92122.40
1,Lanús,Lanús Oeste,49.0,97491.80
2,Lanús,Lanús Este,3.0,13472.88
3,Lanús,Gerli,3.0,2304.48
4,Lanús,Remedios de Escalada,14.0,634875.73
5,Lomas de Zamora,Villa Fiorito,3.0,35016.34
6,Lomas de Zamora,Ingeniero Budge,27.0,3320166.57
7,Lomas de Zamora,Llavallol,15.0,3460396.86
8,Lomas de Zamora,Villa Centenario,9.0,30354.89
9,Lomas de Zamora,Lomas de Zamora,13.0,299000.74


# Datos

In [88]:
print("area", df.loc[(df.limite == 1), 'area_m2'].sum()/1000000, 'Mm²')
print("% area", 100*df.loc[(df.limite == 1), 'area_m2'].sum()/gdf_local_data.to_crs("EPSG:5347").area.sum(), '%')
print("area", df_ep.area_m2.sum()/1000000, 'Mm²')
print("% area", 100*df_ep.area_m2.sum()/df.loc[(df.limite == 1), 'area_m2'].sum(), '%')

area 8.301727590000002 Mm²
% area 8.580924516932031 %
area 7.988954929999999 Mm²
% area 96.2324388916741 %


In [89]:
sup_data = df_ep.groupby(['tipo'])['area_m2'].agg(['size', 'sum'])
sup_data['porc'] = 100* sup_data['sum'] / df_ep.area_m2.sum()
sup_data

,size,sum,porc
tipo,,,
Parque con equipamiento,6,7450274.32,93.257183
Plaza,46,388895.70,4.867917
Plazoleta,125,149784.91,1.874900
